In [82]:
import requests
from bs4 import BeautifulSoup    
import datetime
from decimal import Decimal
import pandas as pd

Function responsible for fetching symbols from main page. URLs from this page let to query historical prices

In [74]:
def fetch_symbols():
    """Fetch symbols table from coinmarketcap.
    
    Returns dictionary in format:
    {
        symbol: {name, symbol, url}
        ...
    }
    """
    main_url = 'https://coinmarketcap.com'

    symbols_dict = {}

    r = requests.get(main_url)

    if r.status_code!=requests.codes.ok:
        print "Error fetching symbols %i: %s" % (r.status_code, r.text)
        r.raise_for_status()
    else:
        doc = BeautifulSoup(r.text, 'html.parser')
        # Look for table with id currencies, should be only one
        currencies_tab = doc.find_all(id='currencies')[0]
        
        for _, tr in enumerate(currencies_tab.select("tbody tr")):
            name = tr['id'].split('-')[1]
        
            l = tr.find_all('td', class_='currency-name')[0].find_all('a')[0]
            symbol = l.text
            url = main_url + l['href']
        
            symbols_dict[symbol] = {'name': name, 'symbol': symbol, 'url': url}
    
    return symbols_dict


Function fetching all historical data for single currency. Require base currency url. In example for bitcoin it will be https://coinmarketcap.com/currencies/bitcoin/ Those URLs are fetched by previous function.

In [79]:
def fetch_historical_data(main_url='https://coinmarketcap.com/currencies/bitcoin/'):
    """Fetch all history of daily prices for a specific currenct.
    
    Params:
    main_url - base url to specific currency"""
    start='20130428'
    
    now = datetime.datetime.now()
    end = "%d%d%d" % (now.year, now.month, now.day)
    
    data_url = main_url+'/historical-data/?start=%s&end=%s'%(start, end)
    r = requests.get(data_url)
    
    result = []
    
    if r.status_code!=requests.codes.ok:
        print "Error fetching data %i: %s" % (r.status_code, r.text)
        r.raise_for_status()
    else:
        doc = BeautifulSoup(r.text, 'html.parser')
        
        data_tab = doc.find_all(id='historical-data')[0].select('table.table')[0]
        
        for _, tr in enumerate(data_tab.select('tbody tr')):
            tds = tr.select('td')
            
            try:
                volume = float(tds[5].text.replace(',',''))
            except ValueError:
                volume = float('nan')
                
            try:
                market_cap = float(tds[6].text.replace(',',''))
            except ValueError:
                market_cap = float('nan')
            
            entry = {
                'date': datetime.datetime.strptime(tds[0].text, '%b %d, %Y').isoformat(),
                'open_price': float(tds[1].text),
                'high_price': float(tds[2].text),
                'low_price': float(tds[3].text),
                'close_price': float(tds[4].text),
                'volume': volume,
                'market_cap': market_cap
            }
            
            result.append(entry)
    
    return result


Example usage of both functions to fetch eth prices and store in csv file

In [99]:
symbols = fetch_symbols()

eth_data = fetch_historical_data(symbols['ETH']['url'])

eth_df = pd.DataFrame(eth_data)

eth_df.to_csv('eth.csv', index=False, columns=['date', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'market_cap'])
